Importy potrzebnych bibliotek i funkcji

In [61]:
from dimacs import *
from math import ceil
from queue import deque, PriorityQueue
import time
import os

Sprawdzarka

In [62]:
def readSolution(graph):
    with open(graph, "r") as f:
        first_line = f.readline()
    return int(first_line.split("=")[1])

def compareResults(file_path, graph_name, method, wrong, show_results=False):
    file_solution = readSolution(file_path)
    algorithm_solution = method(file_path)
    
    if show_results:
        print(f"Graph: {graph_name}")
        print(f"File solution: {file_solution}")
        print(f"Algorithm solution: {algorithm_solution}")
        if file_solution == algorithm_solution:
            print("Results match.")
        else:
            print("Results do not match.")

    return file_solution == algorithm_solution

def testAllGraphs(directory_path, method, show_results=False):
    all_passed = True
    wrong = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            if not compareResults(file_path, filename, method, wrong, show_results):
                all_passed = False
                wrong.append((filename, readSolution(file_path), method(file_path)))

    if all_passed:
        print("All tests passed.")     
    else:
        print("Failed tests:")
        for graph, file_solution, algorithm_solution in wrong:
            print(f"Graph: {graph}")
            print(f"File solution: {file_solution}")
            print(f"Algorithm solution: {algorithm_solution}")
            print()

Dany jest graf nieskierowany G = (V,E), funkcja c: E -> N dająca wagi krawędziom, oraz wyróżnione wierzchołki s i t.
Szukamy scieżki z s do t takiej, że najmniejsza waga krawędzi na tej ścieżce jest jak największa.
Należy zwrócić najmniejszą wagę krawędzi na znalezionej ścieżce.

1. Wykorzystanie struktury find-union

In [63]:
class Node:
    def __init__(self, value):
        self.parent = self
        self.rank = 0
        self.value = value

    def find_set(self):
        if self.parent != self:
            self.parent = self.parent.find_set()
        return self.parent

    def union(self, other):
        root_self = self.find_set()
        root_other = other.find_set()

        if root_self != root_other:
            if root_self.rank > root_other.rank:
                root_other.parent = root_self
            else:
                root_self.parent = root_other
                if root_self.rank == root_other.rank:
                    root_other.rank += 1


def find_union_max_min(graph):
    V, L = loadWeightedGraph(graph)
    nodes = [Node(i) for i in range(V)] 
    L.sort(key=lambda x: x[2], reverse=True)

    for edge in L:
        x = nodes[edge[0]-1]
        y = nodes[edge[1]-1]
        
        if x.find_set() != y.find_set():
            x.union(y)
        if nodes[0].find_set() == nodes[1].find_set():
            return edge[2]

In [64]:
testAllGraphs("graphs", find_union_max_min)

All tests passed.


2. Wyszukiwanie binarne + przegląd grafu metodami BFS/DFS

In [65]:
def binary_search_max_min(graph):

    def convert_graph(V, L):
        G = [set() for _ in range(V)]
        for u, v, c in L: 
            u -= 1
            v -= 1
            G[u].add((v, c))
            G[v].add((u, c)) 
        return G 

    def has_path_with_min_weight(G, min_weight):
        n = len(G)
        visited = [False] * n
        visited[0] = True
        queue = deque()
        queue.append(0)
        while queue:
            vertex = queue.popleft()
            for to_go, weight in G[vertex]:
                if weight >= min_weight and not visited[to_go]: #tak obsluguje nieprzekraczanie min_weight
                    if to_go == 1:
                        return True
                    queue.append(to_go)
                    visited[to_go] = True
        return False

    V, L = loadWeightedGraph(graph)

    max_weight = -float('inf')
    min_weight = float('inf')
    for edge in L:
        max_weight = max(max_weight, edge[2])
        min_weight = min(min_weight, edge[2])

    graph = convert_graph(V, L)
    left = min_weight
    right = max_weight

    while right - left > 1:
        mid = (left + right) / 2
        if has_path_with_min_weight(graph, mid):
            left = mid
        else:
            right = mid
    
    return ceil(left)

In [66]:
testAllGraphs("graphs", binary_search_max_min)

All tests passed.


3. Algorytm a’la Dijkstra

In [67]:
def dijkstra_max_min(graph):

    def convert_graph(V, L):
        G = [set() for _ in range(V)]
        for u, v, c in L: 
            u -= 1
            v -= 1
            G[u].add((v, c))
            G[v].add((u, c)) 
        return G 
    V, L = loadWeightedGraph(graph)
    graph = convert_graph(V, L)

    max_min = [-float('inf')] * V
    max_min[0] = float('inf')

    queue = PriorityQueue()
    queue.put((-float('inf'), 0))

    while not queue.empty():
        weight, u = queue.get()
        if u == 1:
            return max_min[1]

        weight = -weight
        if weight < max_min[u]:
            continue

        for v, w in graph[u]:
            new_weight = min(w, max_min[u]) #kandydat
            if new_weight > max_min[v]:
                max_min[v] = new_weight
                queue.put((-new_weight, v))

In [68]:
testAllGraphs("graphs", dijkstra_max_min)

All tests passed.
